https://en.wikipedia.org/wiki/Unification_(computer_science)

- minikanren , microkanren, faster-minikanren
- harrison https://www.cl.cam.ac.uk/~jrh13/atp/index.html
- traat https://www21.in.tum.de/~nipkow/TRaAT/ but it also has pascal
- pyres
- prover9 - https://github.com/ai4reason/Prover9/blob/cdca95a51d3c3459b8fd2ebbb5ac1504be2172e3/ladr/unify.c#L345
- uniufication fd
- cody's
- ocaml-alg https://github.com/smimram/ocaml-alg/blob/3905b52a90bc6ac7c91054e1f961b8685b77a30a/src/term.ml#L375
- Graham's 
- eprover https://github.com/eprover/eprover/blob/ab3ea0835b13553d3872b858e93739c2b1aeb0e6/TERMS/cte_match_mgu_1-1.c#L463
- vampire https://github.com/vprover/vampire/blob/6b4efd08c39a0fdc5b28f266dc6b639e807903d7/Kernel/RobSubstitution.cpp#L266
- https://eli.thegreenplace.net/2018/unification/

http://www.cs.man.ac.uk/~hoderk/ubench/unification_full.pdf Comparing Unification Algorithms in First-Order
Theorem Proving

https://users.soe.ucsc.edu/~lkuper/papers/walk.pdf Efficient representations for triangular substitutions: A comparison in miniKanren


https://www.proquest.com/openview/f8298f16044b130532ceca70ed4d60c2/1?pq-origsite=gscholar&cbl=2026366&diss=y Automated Theorem Proving in Higher-Order Logic - Bhayat thesis

do on z3

https://www.philipzucker.com/resolution1/


E-unificationun
https://web.archive.org/web/20200211213303id_/https://publikationen.sulb.uni-saarland.de/bitstream/20.500.11880/25047/1/RR_92_01.pdf

HO-unification as a alngague. When you have full HO unfication, do you need prolog search anymore? You already have so much nontdeterminis


uniification  as observation
unification as fixed point - When written as an inference system this espcailly makes sense.


Lazy vs eager substitution in unification

Tringular = Lazy

Eager vs lazy normalization of union find.

Recursive vs loop.
We have access to the "todo" equations in loop form in the form of some kind of stack.


Inference rule form.
I kind of feel like completion you're moving from E to S, building a valid substitution. E;S. 
Inference rule form is closer to loop form. You have access to all of E.
Typical loop form just pops, but you could use a loop to find the next one to deal with.



Logical Form.
Exists([x,y,z], T) == Exist([x,y,z], T)
Exists([], T1 = T2)
ForAll([], ) # rigid.

occursw check vs scope escape. That prolog trick
Just having Exists and ForAll might give you a variation of miller. Ignore lambda?





# Z3 Unify


Unification is useful for question asking (exists)
and for inferring new facts

questions ~ goals ~ queries ~ BoolRef ~ typed at prolog repl ~ prolog call
answers ~ solutions ~ Proof ~ returned to prolog repl ~ prolog return

questions can be forall or exists
answers can also be forall or exists




In [ ]:
def unify_generalized(t):
    """
    Miller nadathur book
    alternatig quatifiers then conjuction of equations
    Q Q Q Q Q (t1 =t2 /\ t3 = t4 /\ t5 = t6 /\ t7 = t8 /\ t9 = t10)

    """
    vars = []
    while is_quantifier(t)
        vars.append(t.vars)
        t = body()
    todo = [t]
    eqs = []
    while todo:
        t = todo.pop()
        if is_eq(t):
            eqs.append(t)
        elif is_conj(t):
            todo.extend(t.children())
        else:
            raise ValueError("not a conjunction of equations")
    subst = {}
    for eq in eqs:
        if eq.lhs in subst:
            if subst[eq.lhs] != eq.rhs:
                raise ValueError("inconsistent equations")
            # scope check
        else:
            subst[eq.lhs] = eq.rhs

    

In [ ]:
def unify_prove(t : z3.BoolRef):
    """
    request an exists formula
    return a forall _Proof_.
    """
    if is_quantifier(t):
        assert t.is_exists():
    

def unify_resolve(q : kd.Proof, t : kd.Proof):
    # Take two forall proofs q and t, and return a proof of the resolution of q and t
    # forall forall -> forall
def unify_apply(t : BoolRef, t : kd.Proof):
    # exist goal + forall proof -> (exist goal', Proof of exists goal' -> exists goal)
    # can just use matching?
    # forall goal + forall proof -> (forall goal', Proof of forall (goal' -> goal)



In [ ]:
from z3 import *

def subterms(t):
    ts = set(t)
    for c in t.children():
        ts |= subterms(c)
    return ts

def unify(t1 : ExprRef, t2 : ExprRef, subst, vs):
    if t1.eq(t2):
        return
    if t2 in vs:
        t2, t1 = t1, t2
    if t1 in vs:
        if t1 in subst:
            unify(subst[t1], t2, subst, vs)
        else:
            if t1 in subterms(t2): # occurs
                raise Exception("Unification failed")
            subst = {i : substitute(k, t1, t2) for i,k in subst.items()} # eager subst. Oh but I can't normalize all my terms too
            subst[t1] = t2
    else:
        if t1.decl() == t2.decl():
            for  c1,c2 in zip(t1.children(), t2.children()):
                unify(c1, c2, subst, vs)
        else:
            raise Exception("Unification failed")
    else:
        raise Exception("Unification failed")
    

unify()


In [20]:
type(Var(0, StringSort()))
x = Int('x')
is_app(x)

True

In [21]:
from z3 import *
def z3_match(t : ExprRef, pat : ExprRef) -> dict[z3.ExprRef, z3.ExprRef]:
    """
    Pattern match t against pat. Variables are constructed as `z3.Var(i, sort)`.
    Returns substitution dict if match succeeds.
    Returns None if match fails.
    Outer quantifier (Exists, ForAll, Lambda) in pat is ignored.
    """
    if z3.is_quantifier(pat):
        pat = pat.body()
    subst = {}
    todo = [(t,pat)]
    while len(todo) > 0:
        t,pat  = todo.pop()
        if t.eq(pat):
            continue
        if z3.is_var(pat):
            if pat in subst:
                if not subst[pat].eq(t):
                    return None
            else:
                subst[pat] = t
        elif z3.is_app(t) and z3.is_app(pat):
            if pat.decl() == t.decl():
                todo.extend(zip(t.children(), pat.children()))
            else:
                return None
        else:
            raise Exception("Unexpected subterm or subpattern", t, pat)
    return subst
# not worth doing
def match_quant(t,p):
    if is_quantifier(p):
        return z3_match(t,p.body())
    else:
        raise Exception("Not a quantifier")
R = RealSort()
x,y,z = Reals('x y z')
z3_match(x, Var(0, R))
pmatch(x + y, Var(0, R) + Var(1, R))
pmatch(x + y, Var(0, R) + Var(1, R) + Var(2, R))
pmatch(x + y + x, Var(0, R) + Var(1, R) + Var(0, R))
pmatch(x + y + x*6, Var(0, R) + Var(1, R) + Var(2, R))

match_quant(x + y + x*6 == 0, ForAll([x,y,z], x + y + z == 0))

# If I wanted deeper matching into binders, then what?

{Var(0): x*6, Var(1): y, Var(2): x}

# E-unify Egraph

In [ ]:
# E-unification Egraph Style
from dataclasses import dataclass
from typing import Any
@dataclass
class Unify():
    uf: list[int]
    obs: dict[int, tuple[Any, tuple[int]]]
    def __init__(self):
        self.uf = []
        self.obs = {}
    def make_var(self):
        v = len(self.uf)
        self.uf.append(v)
        return v
    def add_term(self, t):
        # try to check for rationality?
        if isinstance(t, int):
            return t
        head, *args = t
        args = tuple(self.add_term(arg) for arg in args)
        # we're not interning at all here, which is fine but inefficient.
        x = self.make_var() 
        self.observe(x, (head, args))
        return x
    def find(self, x):
        while self.uf[x] != x:
            x = self.uf[x]
        return x
    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)
        self.uf[x] = y
        return y
    def merge_obs(self, obs, obs1): # unify_flat
        head, args = obs
        head1, args1 = obs1
        if head != head1:
            raise Exception("head mismatch")
        if len(args) != len(args1):
            raise Exception("arity mismatch")
        for a,a1 in zip(args, args1):
            self.union(a,a1)
    def observe(self, x, obs):
        x = self.find(x)
        o = self.obs.get(x)
        if o != None:
            self.merge_obs(o, obs)
        self.obs[x] = obs
    def extract(self,x):
        # assuming a well founded term. Could extract the rational term?
        x = self.find(x)
        f = self.obs.get(x)
        if f == None:
            return x
        else:
            head, args = f
            return (head, *[self.extract(a) for a in args])
    def rebuild(self):
        while True:
            newobs = {}
            # "unobserved" vars aren't be merged. unobserved is actually Maximally observed, i.e. has identity.
            for x,obs in self.obs.items():
                x1 = self.find(x)
                obs1 = self.obs.get(x1)
                if obs1 != None:
                    self.merge_obs(obs, obs1)
                (head, args) = obs
                newobs[x1] = (head, tuple(map(self.find, args)))
            if self.obs == newobs:
                break
            self.obs = newobs

U = Unify()
x = U.make_var()
y = U.make_var()
z = U.make_var()
print(U)
#U.union(x,y)
a_x = U.add_term(("a", ("b", z)))
b_y = U.add_term(("a", y))
U.union(a_x, b_y)
print(U)
U.rebuild()
print(U)
U.extract(a_x)

# Example Unification algorithms
## PyRes

In [ ]:
# https://github.com/eprover/PyRes/blob/master/unification.py
def occursCheck(x, t):
   """
   Perform an occurs-check, i.e. determine if the variable x occurs in
   the term t. If that is the case (and t != x), the two can never be
   unified.
   """
   if termIsCompound(t):
        for i in t[1:]:
            if occursCheck(x, i):
                return True
        return False
   else:
       return x == t


def mguTermList(l1, l2, subst):
    """
    Unify all terms in l1 with the corresponding terms in l2 with a
    common substitution variable "subst". We don't use explicit
    equations or pairs of terms here - if l1 is [s1, s2, s3] and l2 is
    [t1, t2, t3], this represents the set of equations {s1=t1, s2=t2,
    s3=t3}. This makes several operations easier to implement.
    """
    assert len(l1)==len(l2)
    while(len(l1)!=0):
       # Pop the first term pair to unify off the lists (pop removes
       # and returns the denoted elements)
       t1 = l1.pop(0)
       t2 = l2.pop(0)
       if termIsVar(t1):
          if t1==t2:
             # This implements the "Solved" rule.
             # We could always test this upfront, but that would
             # require an expensive check every time.
             # We descend recursively anyway, so we only check this on
             # the terminal case.
             continue
          if occursCheck(t1, t2):
             return None
          # Here is the core of the "Bind" rule
          # We now create a new substitution that binds t2 to t1, and
          # apply it to the remaining unification problem. We know
          # that every variable will only ever be bound once, because
          # we eliminate all occurances of it in this step - remember
          # that by the failed occurs-check, t2 cannot contain t1.
          new_binding = Substitution([(t1, t2)])
          l1 = [new_binding(t) for t in l1]
          l2 = [new_binding(t) for t in l2]
          subst.composeBinding((t1, t2))
       elif termIsVar(t2):
          # Symmetric case
          # We know that t1!=t2, so we can drop this check
          if occursCheck(t2, t1):
             return None
          new_binding = Substitution([(t2, t1)])
          l1 = [new_binding(t) for t in l1]
          l2 = [new_binding(t) for t in l2]
          subst.composeBinding((t2, t1))
       else:
          assert termIsCompound(t1) and termIsCompound(t2)
          # Try to apply "Decompose"
          # For f(s1, ..., sn) = g(t1, ..., tn), first f and g have to
          # be equal...
          if terms.termFunc(t1) != terms.termFunc(t2):
             # Nope, "Structural fail":
             return None
          # But if the symbols are equal, here is the decomposition:
          # We need to ensure that for all i si=ti get
          # added to the list of equations to be solved.
          l1.extend(termArgs(t1))
          l2.extend(termArgs(t2))
    return subst


def mgu(t1, t2):
    """
    Try to unify t1 and t2, return substitution on success, or None on failure.
    """
    res =  mguTermList([t1], [t2], Substitution())
    res2 = "False"
    if res:
       res2 = "True"
    # print("% :", term2String(t1), " : ", term2String(t2), " => ", res2);
    return res

## Minikanren

In [ ]:
%file /tmp/minikanren_unif.scm
;; http://webyrd.net/scheme-2013/papers/HemannMuKanren2013.pdf
;; https://github.com/jasonhemann/microKanren/blob/master/microKanren.scm

; Jason Hemann and Dan Friedman
;; microKanren, final implementation from paper

(define (var c) (vector c))
(define (var? x) (vector? x))
(define (var=? x1 x2) (= (vector-ref x1 0) (vector-ref x2 0)))

(define (walk u s)
  (let ((pr (and (var? u) (assp (lambda (v) (var=? u v)) s))))
    (if pr (walk (cdr pr) s) u)))

(define (ext-s x v s) `((,x . ,v) . ,s))

(define (== u v)
  (lambda (s/c)
    (let ((s (unify u v (car s/c))))
      (if s (unit `(,s . ,(cdr s/c))) mzero))))

(define (unit s/c) (cons s/c mzero))
(define mzero '())

(define (unify u v s)
  (let ((u (walk u s)) (v (walk v s)))
    (cond
      ((and (var? u) (var? v) (var=? u v)) s)
      ((var? u) (ext-s u v s))
      ((var? v) (ext-s v u s))
      ((and (pair? u) (pair? v))
       (let ((s (unify (car u) (car v) s)))
         (and s (unify (cdr u) (cdr v) s))))
      (else (and (eqv? u v) s)))))


## Harrison

In [ ]:
%%file /tmp/harrison.ml
(* ========================================================================= *)
(* Unification for first order terms.                                        *)
(*                 https://www.cl.cam.ac.uk/~jrh13/atp/OCaml/unif.ml                                                           *)
(* Copyright (c) 2003-2007, John Harrison. (See "LICENSE.txt" for details.)  *)
(* ========================================================================= *)

let rec istriv env x t =
  match t with
    Var y -> y = x or defined env y & istriv env x (apply env y)
  | Fn(f,args) -> exists (istriv env x) args & failwith "cyclic";;

(* ------------------------------------------------------------------------- *)
(* Main unification procedure                                                *)
(* ------------------------------------------------------------------------- *)

let rec unify env eqs =
  match eqs with
    [] -> env
  | (Fn(f,fargs),Fn(g,gargs))::oth ->
        if f = g & length fargs = length gargs
        then unify env (zip fargs gargs @ oth)
        else failwith "impossible unification"
  | (Var x,t)::oth | (t,Var x)::oth ->
        if defined env x then unify env ((apply env x,t)::oth)
        else unify (if istriv env x t then env else (x|->t) env) oth;;

(* ------------------------------------------------------------------------- *)
(* Solve to obtain a single instantiation.                                   *)
(* ------------------------------------------------------------------------- *)

let rec solve env =
  let env' = mapf (tsubst env) env in
  if env' = env then env else solve env';;

(* ------------------------------------------------------------------------- *)
(* Unification reaching a final solved form (often this isn't needed).       *)
(* ------------------------------------------------------------------------- *)

let fullunify eqs = solve (unify undefined eqs);;

## Prover9


In [ ]:
%%file /tmp/prover9.c
BOOL unify(Term t1, Context c1,
           Term t2, Context c2, Trail *trp)
{
  Trail tpos, tp, t3;
  int vn1, vn2;

  DEREFERENCE(t1, c1)  /* dereference macro */

  DEREFERENCE(t2, c2)  /* dereference macro */

  /* Now, neither t1 nor t2 is a bound variable. */

  if (VARIABLE(t1)) {
    vn1 = VARNUM(t1);
    if (VARIABLE(t2)) {
      /* both t1 and t2 are variables */
      if (vn1 == VARNUM(t2) && c1 == c2)
        return TRUE;  /* identical */
      else {
        BIND_TR(vn1, c1, t2, c2, trp)
        return TRUE;
      }
    }
    else {
      /* t1 variable, t2 not variable */
      if (occur_check(vn1, c1, t2, c2)) {
        BIND_TR(vn1, c1, t2, c2, trp)
        return TRUE;
      }
      else
        return FALSE;  /* failed occur_check */
    }
  }

  else if (VARIABLE(t2)) {
    /* t2 variable, t1 not variable */
    vn2 = VARNUM(t2);
    if (occur_check(vn2, c2, t1, c1)) {
      BIND_TR(vn2, c2, t1, c1, trp)
      return TRUE;
    }
    else
      return FALSE;  /* failed occur_check */
  }
    
  else if (SYMNUM(t1) != SYMNUM(t2))
    return FALSE;  /* fail because of symbol clash */

  else if (ARITY(t1) == 0)
    return TRUE;

  else {  /* both complex with same symbol */
    int i, arity;

    tpos = *trp;  /* save trail position in case of failure */
        
    i = 0; arity = ARITY(t1);
    while (i < arity && unify(ARG(t1,i), c1, ARG(t2,i), c2, trp))
      i++;

    if (i == arity)
      return TRUE;
    else {  /* restore trail and fail */
      tp = *trp;
      while (tp != tpos) {
        tp->context->terms[tp->varnum] = NULL;
        tp->context->contexts[tp->varnum] = NULL;
        t3 = tp;
        tp = tp->next;
        free_trail(t3);
      }
      *trp = tpos;
      return FALSE;
    }
  }
}  /* unify */

## Ocaml alg

In [ ]:
%%file /tmp/ocaml_alg.ml
(* https://github.com/smimram/ocaml-alg/blob/main/src/term.ml#L375 *)
(** Most general unifier. *)
let unify t1 t2 =
  (* Printf.printf "UNIFY %s WITH %s\n%!" (to_string t1) (to_string t2); *)
  let rec aux q s =
    match q with
    | [] -> s
    | p::q ->
       match p with
       | Var x, t ->
          if occurs x t then raise Not_unifiable;
          let s' = Subst.simple x t in
          let f = Subst.app s' in
          let q = List.map (fun (t1,t2) -> f t1, f t2) q in
          let s = Subst.compose s s' in
          aux q (Subst.add s x t)
       | t, Var x -> aux ((Var x,t)::q) s
       | App (f1,a1), App (f2,a2) ->
          if not (Op.eq f1 f2) then raise Not_unifiable;
          let q = (List.map2 pair a1 a2) @ q in
          aux q s
  in
  let s = aux [t1,t2] Subst.empty in
  assert (eq (Subst.app s t1) (Subst.app s t2));
  s

## TRAAT

In [ ]:
%%file /tmp/traat.ml
(*** https://www21.in.tum.de/~nipkow/TRaAT/programs/trs.ML The basics of term rewriting: terms, unification, matching, normalization

ML Programs from Chapter 4 of

  Term Rewriting and All That
  by Franz Baader and Tobias Nipkow,
  (Cambridge University Press, 1998)

Copyright (C) 1998 by Cambridge University Press.
Permission to use without fee is granted provided that this copyright
notice is included in any copy.
***)

type vname = string * int;

datatype term = V of vname | T of string * term list;

(* indom: vname -> subst -> bool *)
fun indom x s = exists (fn (y,_) => x = y) s;

(* app: subst -> vname -> term *)
fun app ((y,t)::s) x  =  if x=y then t else app s x;

(* lift: subst -> term -> term *)
fun lift s (V x)    = if indom x s then app s x else V x
  | lift s (T(f,ts)) = T(f, map (lift s) ts);

(* occurs: vname -> term -> bool *)
fun occurs x (V y)    = x=y
  | occurs x (T(_,ts)) = exists (occurs x) ts;

exception UNIFY;

(* solve: (term * term)list * subst -> subst *)
fun solve([], s) = s
  | solve((V x, t) :: S, s) =
      if V x = t then solve(S,s) else elim(x,t,S,s)
  | solve((t, V x) :: S, s) = elim(x,t,S,s)
  | solve((T(f,ts),T(g,us)) :: S, s) =
      if f = g then solve(zip(ts,us) @ S, s) else raise UNIFY

(* elim: vname * term * (term * term) list * subst -> subst *)
and elim(x,t,S,s) =
      if occurs x t then raise UNIFY
      else let val xt = lift [(x,t)]
           in solve(map (fn (t1,t2) => (xt t1, xt t2)) S,
                    (x,t) :: (map (fn (y,u) => (y, xt u)) s))
           end;

(* unify: term * term -> subst *)
fun unify(t1,t2) = solve([(t1,t2)], []);

(* matchs: (term * term) list * subst -> subst *)
fun matchs([], s) = s
  | matchs((V x, t) :: S, s) =
      if indom x s then if app s x = t then matchs(S,s) else raise UNIFY
      else matchs(S,(x,t)::s)
  | matchs((t, V x) :: S, s) = raise UNIFY
  | matchs((T(f,ts),T(g,us)) :: S, s) =
      if f = g then matchs(zip(ts,us) @ S, s) else raise UNIFY;

(* match: term * term -> subst *)
fun match(pat,obj) = matchs([(pat,obj)],[]);

exception NORM;

(* rewrite: (term * term) list -> term -> term *)
fun rewrite [] t = raise NORM
  | rewrite ((l,r)::R) t = lift(match(l,t)) r
                          handle UNIFY => rewrite R t;

(* norm: (term * term) list -> term -> term *)
fun norm R (V x) = V x
  | norm R (T(f,ts)) =
      let val u = T(f, map (norm R) ts)
      in (norm R (rewrite R u))  handle NORM => u  end;

alpha prolog or lambda prolog as a librtary would be good.
alpha leantap
(an interpreter)


In [ ]:
import janus_swi as janus

# leantap with higher order unify?
# trs with higher order unify?

janus.consult("hounify", """

ho_unify(bvar()).
ho_unify(app(F1,X1), app(F2,X2)) :- ho_unify(F1,F2, Sig), ho_unify(X1,X2,Sig).
ho_unify(lam(X1,B1), lam(X2,B2), Sig) :- gensym(S), V =.. [S|Sig], X1 = V, X2 = V, ho_unify(B1,B2,Sig).

              
% noiminal_unify(T1, T2 ,  Perm)
nominal(atom(A), atom(B), Perm1, Perm2) :- member(A-B, Perm1), Perm1 = Perm2.
nominal(atom(A), atom(B), Perm1, Perm2) :- +/ member(A-Y, Perm1), +/ member(Z-B, Perm1), Perm2 = [A-B|Perm1].
nominal(app(F,Args), app(F1,Args2), Perm1, Perm2) :- nominal(F,F1)
nominal

% maybe a dcg of the permutation? We are threading a state.
nominal(atom(A), atom(B)) --> 

% https://stackoverflow.com/questions/64638801/the-unification-algorithm-in-prolog#:~:text=unify(A%2CB)%3A%2D%20atomic,B)%2CA%3DB.        
              unify(A,B):-
   atomic(A),atomic(B),A=B.
unify(A,B):-
   var(A),A=B.            % without occurs check
unify(A,B):-
   nonvar(A),var(B),A=B.  % without occurs check
unify(A,B):-
   compound(A),compound(B),
   A=..[F|ArgsA],B=..[F|ArgsB],
   unify_args(ArgsA,ArgsB).
   
unify_args([A|TA],[B|TB]):-
   unify(A,B),
   unify_args(TA,TB).
unify_args([],[]).
""")

bottom up ho unify is simpler. Dowek. Maybe anmm inverse method/magic set is in order.
